In [35]:
!pip install keras-tuner

In [36]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [37]:
fashion_mnist=keras.datasets.fashion_mnist

In [38]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [39]:
train_images=train_images/255.0
test_images=test_images/255.0

In [40]:
train_images[0].shape

(28, 28)

In [41]:
train_images=train_images.reshape(len(train_images),28,28,1) 
test_images=test_images.reshape(len(test_images),28,28,1) 

In [45]:
def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                                              kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                                              activation='relu',
                                              input_shape=(28,28,1)
                                              ),
                          keras.layers.Conv2D(filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                                              kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                                               activation='relu'
                                               ),
                          keras.layers.Flatten(),
                          keras.layers.Dense(
                              units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                              activation='relu'
                          ),
                          keras.layers.Dense(10,activation='softmax')
                          ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [46]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [49]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

In [51]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 2 Complete [00h 13m 45s]
val_accuracy: 0.9101666808128357

Best val_accuracy So Far: 0.9101666808128357
Total elapsed time: 00h 14m 14s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |48                |32                
conv_1_kernel     |5                 |3                 
conv_2_filter     |48                |64                
conv_2_kernel     |5                 |5                 
dense_1_units     |112               |96                
learning_rate     |0.001             |0.001             

Epoch 1/3
  27/1688 [..............................] - ETA: 4:08 - loss: 1.4902 - accuracy: 0.5185

KeyboardInterrupt: ignored

In [52]:
model=tuner_search.get_best_models(num_models=1)[0]

In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        51264     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2973792   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,026,346
Trainable params: 3,026,346
Non-trainable params: 0
_________________________________________________________________
